In [1]:
import tensorflow as tf
# getting data directly from a tensorboard log dir
from tensorflow.python.summary import event_multiplexer

import pandas as pd

from collections import OrderedDict

# from matplotlib import pyplot as plt
# from matplotlib.colors import Normalize
# from numpy import around

import os

In [2]:
"""
Scalars tags (not all but most of them):
Mean_Reciprocal_Rank/Mean_Reciprocal_Rank_test
Accuracy/Accuracy_test
In_top_5/In_top_5_test
Cost_function/Total_cost_test
Cost_function_additional_metrics/Cross_entropy_test

attributes
wall_time
step
value
"""

# select first directory (as example)
# log_dir = child_dir[0]

def get_optimal_event(scalars, selection_func):
    """
    Inputs are list of ScalarEvent and a selection function (either min or max).
    e.g. for accuracy, max should be selected, while for cost, min is appropriate.
    Returns the "optimal" event, i.e. the event where the optimal value was achieved.
    """
    if selection_func not in (min, max):
        raise ValueError('selection_func should be either min or max, got unexpected {}'
                         .format(selection_func.__name__))
    
    # optimal value
    optimal_val = selection_func(
        [event.value for event in scalars])
    # optimal event (corresponding to the optimal value)
    optimal_event = [event for event in scalars 
                     if event.value == optimal_val][0]
    
    return optimal_event

In [3]:
def get_all_optimals(log_dir):
    """
    Input is a log_dir (e.g. as from child_dir)
    Gets optimal events (using 'get_optimal_event') for 
        total cost, Mean Reciprocal Rank and accuracy.
    Returns the three optimal events (optimal_cost, optimal_mrr, optimal_accuracy),
        as well as the three ScalarEvent lists (total_cost, mrr, accuracy)
    """
    # get ScalarEvent lists
    total_cost = ea.Scalars(log_dir, 'Cost_function/Total_cost_test')
    mrr = ea.Scalars(log_dir, 'Mean_Reciprocal_Rank/Mean_Reciprocal_Rank_test')
    accuracy = ea.Scalars(log_dir, 'Accuracy/Accuracy_test')

    # get optimal events
    optimal_cost = get_optimal_event(total_cost, min)
    optimal_mrr = get_optimal_event(mrr, max)
    optimal_accuracy = get_optimal_event(accuracy, max)

    return (optimal_cost, optimal_mrr, optimal_accuracy, 
            total_cost, mrr, accuracy)

In [4]:
def value_at_other_optimal(scalars, optimal_event):
    """
    Inputs are scalars, a list of ScalarEvent and 
        optimal_event, an optimal ScalarEvent.
    Retruns the value from scalars at the optimal_event step.
    e.g. value_at_other_optimal(accuracy, optimal_cost) returns
        the accuracy value at the step where cost was optimal.
    """
    return [event.value for event in scalars 
            if event.step == optimal_event.step][0]

In [5]:
def metrics_dict_from_log(log_dir, 
                          optimal_cost=None, 
                          optimal_mrr=None, 
                          optimal_accuracy=None, 
                          total_cost=None, 
                          mrr=None, 
                          accuracy=None):
    """
    Input log_dir is a Tensorboard log directory.
    Other inputs are optional and will be generated if not provided.
    Returns an OrderedDict with the model string (log dir name) and evaluation metrics.
    """
    # check if all optional input values are None
    if not any(a is not None 
               for a in [optimal_cost, 
                         optimal_mrr, 
                         optimal_accuracy, 
                         total_cost, 
                         mrr, 
                         accuracy]):
        # get evaluation metrics data from log dir
        (optimal_cost, optimal_mrr, optimal_accuracy, 
         total_cost, mrr, accuracy) = get_all_optimals(log_dir)
        
    return OrderedDict(
        [('Model_str', log_dir), 
         ('Cost @ optimal cost', value_at_other_optimal(total_cost, optimal_cost)), 
         ('MRR @ optimal cost', value_at_other_optimal(mrr, optimal_cost)), 
         ('Accuracy @ optimal cost', value_at_other_optimal(accuracy, optimal_cost)), 
         ('Cost @ optimal MRR', value_at_other_optimal(total_cost, optimal_mrr)), 
         ('MRR @ optimal MRR', value_at_other_optimal(mrr, optimal_mrr)), 
         ('Accuracy @ optimal MRR', value_at_other_optimal(accuracy, optimal_mrr)), 
         ('Cost @ optimal accuracy', value_at_other_optimal(total_cost, optimal_accuracy)), 
         ('MRR @ optimal accuracy', value_at_other_optimal(mrr, optimal_accuracy)), 
         ('Accuracy @ optimal accuracy', value_at_other_optimal(accuracy, optimal_accuracy))
         ])

In [6]:
# specify path (for parent log dir)
log_parent_dir = './experiments/logdir_exper_4_20_GRU/'

In [7]:
ea = event_multiplexer.EventMultiplexer().AddRunsFromDirectory(log_parent_dir)
ea.Reload()  # load

INFO:tensorflow:Event Multiplexer initializing.
INFO:tensorflow:Event Multiplexer done initializing
INFO:tensorflow:Starting AddRunsFromDirectory: ./experiments/logdir_exper_4_20_GRU/
INFO:tensorflow:Adding events from directory ./experiments/logdir_exper_4_20_GRU/GRU,bidir=F,noisy_tanh,learn_p=T,noise_alpha=1.15,noise_half_normal=F,keep_infreq_labels=F,learn_rate=1.0E-02,keep_prob=0.7,one_hot,hidden_state_size=128,l2_wieght_reg=1.0E-03,target_rep_weight=0.3
INFO:tensorflow:Constructing EventAccumulator for ./experiments/logdir_exper_4_20_GRU/GRU,bidir=F,noisy_tanh,learn_p=T,noise_alpha=1.15,noise_half_normal=F,keep_infreq_labels=F,learn_rate=1.0E-02,keep_prob=0.7,one_hot,hidden_state_size=128,l2_wieght_reg=1.0E-03,target_rep_weight=0.3
INFO:tensorflow:Adding events from directory ./experiments/logdir_exper_4_20_GRU/GRU,bidir=F,noisy_tanh,learn_p=T,noise_alpha=0.9,noise_half_normal=F,keep_infreq_labels=F,learn_rate=1.0E-02,keep_prob=0.7,one_hot,hidden_state_size=64,l2_wieght_reg=1.0E-0

In [8]:
# get a list of all subfolders in the parent log dir
child_dir = next(os.walk(log_parent_dir))[1]

In [9]:
parent_dir_metrics = [metrics_dict_from_log(log_dir)
                      for log_dir in child_dir]

In [10]:
exper_metrics = pd.DataFrame(parent_dir_metrics)
exper_metrics.set_index(keys='Model_str', inplace=True, verify_integrity=True)
# exper_metrics.drop(labels='')

In [11]:
exper_metrics.head(n=5)

,Cost @ optimal cost,MRR @ optimal cost,Accuracy @ optimal cost,Cost @ optimal MRR,MRR @ optimal MRR,Accuracy @ optimal MRR,Cost @ optimal accuracy,MRR @ optimal accuracy,Accuracy @ optimal accuracy
Model_str,,,,,,,,,
"GRU,bidir=F,noisy_tanh,learn_p=T,noise_alpha=1.15,noise_half_normal=F,keep_infreq_labels=F,learn_rate=1.0E-02,keep_prob=0.7,one_hot,hidden_state_size=128,l2_wieght_reg=1.0E-03,target_rep_weight=0.3",0.589201,0.940727,0.923288,0.643311,0.947434,0.934247,0.640226,0.947393,0.934247
"GRU,bidir=F,noisy_tanh,learn_p=T,noise_alpha=0.9,noise_half_normal=F,keep_infreq_labels=F,learn_rate=1.0E-02,keep_prob=0.7,one_hot,hidden_state_size=64,l2_wieght_reg=1.0E-04,target_rep_weight=0.3",0.474603,0.951148,0.926027,0.479378,0.952482,0.928767,0.499065,0.950281,0.931507
"GRU,bidir=F,noisy_tanh,learn_p=F,noise_alpha=1.15,noise_half_normal=T,keep_infreq_labels=F,learn_rate=1.0E-02,keep_prob=0.7,one_hot,hidden_state_size=128,l2_wieght_reg=1.0E-04,target_rep_weight=0.3",0.534581,0.948121,0.926027,0.563338,0.948638,0.936986,0.575473,0.948103,0.936986
"GRU,bidir=F,noisy_tanh,learn_p=F,noise_alpha=1.15,noise_half_normal=T,keep_infreq_labels=F,learn_rate=1.0E-02,keep_prob=0.7,one_hot,hidden_state_size=128,l2_wieght_reg=1.0E-03,target_rep_weight=0.3",0.442546,0.947873,0.928767,0.536976,0.955174,0.942466,0.536976,0.955174,0.942466
"GRU,bidir=F,noisy_tanh,learn_p=F,noise_alpha=1.15,noise_half_normal=F,keep_infreq_labels=F,learn_rate=1.0E-02,keep_prob=0.7,one_hot,hidden_state_size=64,l2_wieght_reg=1.0E-03,target_rep_weight=0.3",0.744216,0.938824,0.923288,0.747115,0.944983,0.928767,0.752778,0.943221,0.931507


In [12]:
exper_metrics.style.background_gradient(
    cmap='cool', low=.5, high=0).highlight_null('red')